Notebook from networkx

# Community Detection using Girvan-Newman

This example shows the detection of communities in the Zachary Karate
Club dataset using the Girvan-Newman method.

We plot the change in modularity as important edges are removed.
Graph is coloured and plotted based on community detection when number
of iterations are 1 and 4 respectively.


In [1]:
from pyspark.sql import SparkSession
import sparknlp
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

spark = SparkSession \
        .builder \
        .appName("network") \
        .getOrCreate()

24/11/25 16:36:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
#Asked ChatGPT: How can I create a graph from two csv files? (one with nodes and one with edges)

# File paths (replace with your file paths)
nodes_file = "vertices/vertices.csv"  # CSV with one column, no header
edges_file = "edges/edges.csv"  # CSV with two columns, no header

# Load nodes
nodes_df = pd.read_csv(nodes_file, header=None, names=["node"])
nodes = nodes_df["node"].tolist()  # Convert to a list of nodes

# Load edges
edges_df = pd.read_csv(edges_file, header=None, names=["source", "target"])
edges = edges_df.values.tolist()  # Convert to a list of edge tuples

In [3]:
# Create graph
G = nx.DiGraph()  
G.add_nodes_from(nodes)  
G.add_edges_from(edges) 

In [4]:
communities = nx.community.girvan_newman(G)

In [5]:
communities = list(communities)

KeyboardInterrupt: 

In [ ]:
# Modularity -> measures the strength of division of a network into modules
modularity_df = pd.DataFrame(
    [
        [k + 1, nx.community.modularity(G, communities[k])]
        for k in range(len(communities))
    ],
    columns=["k", "modularity"],
)




In [ ]:
# function to create node colour list
def create_community_node_colors(graph, communities):
    number_of_colors = len(communities)
    colors = ["#D4FCB1", "#CDC5FC", "#FFC2C4", "#F2D140", "#BCC6C8"][:number_of_colors]
    node_colors = []
    for node in graph:
        current_community_index = 0
        for community in communities:
            if node in community:
                node_colors.append(colors[current_community_index])
                break
            current_community_index += 1
    return node_colors


# function to plot graph with node colouring based on communities
def visualize_communities(graph, communities, i):
    node_colors = create_community_node_colors(graph, communities)
    modularity = round(nx.community.modularity(graph, communities), 6)
    title = f"Community Visualization of {len(communities)} communities with modularity of {modularity}"
    pos = nx.spring_layout(graph, k=0.3, iterations=50, seed=2)
    plt.subplot(3, 1, i)
    plt.title(title)
    nx.draw(
        graph,
        pos=pos,
        node_size=1000,
        node_color=node_colors,
        with_labels=True,
        font_size=20,
        font_color="black",
    )


fig, ax = plt.subplots(3, figsize=(15, 20))

# Plot graph with colouring based on communities
visualize_communities(G, communities[0], 1)
visualize_communities(G, communities[3], 2)

# Plot change in modularity as the important edges are removed
modularity_df.plot.bar(
    x="k",
    ax=ax[2],
    color="#F2D140",
    title="Modularity Trend for Girvan-Newman Community Detection",
)
plt.show()